# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Table of Contents

 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#cf): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#utilization-rate): Utilization of servicing equipment
 - [Number of Tows](#tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-costs): Total fixed costs
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [PySAM-Powered Results](#pysam): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 96.1%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.94,0.98,0.97,0.93,0.94,0.94,0.98,0.98,0.97,0.96,0.98,0.96,0.92,0.98,0.98,0.98,0.98,0.98,0.98,0.94,0.95,0.95,0.98,0.98,0.94,0.98,0.92,0.98,0.97,0.92,0.97,0.94,0.98,0.98,0.98,0.98,0.98,0.98,0.97,0.98,0.95,0.97,0.97,0.93,0.98,0.93,0.98,0.98,0.94,0.98,0.98,0.97,0.85,0.98,0.98,0.97,0.98,0.97,0.98,0.98,0.97,0.93,0.96,0.96,0.98,0.96,0.95,0.90,0.93,0.90,0.98,0.98,0.98,0.85,0.98,0.98,0.91,0.96,0.98,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.97
2004,0.96
2005,0.94
2006,0.97
2007,0.96
2008,0.96
2009,0.96
2010,0.96
2011,0.96


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.96
3,0.96
4,0.96
5,0.96
6,0.96
7,0.95
8,0.97
9,0.97


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.99
     3          0.98
     4          0.97
     5          0.96
     6          0.94
     7          0.94
     8          0.99
     9          0.99
     10         0.96
     11         0.95
     12         0.95
2004 1          0.97
     2          0.97
     3          0.98
     4          0.98
     5          0.97
     6          0.96
     7          0.97
     8          0.97
     9          0.95
     10         0.95
     11         0.95
     12         0.95

<a id="cf"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.45,0.47,0.46,0.45,0.45,0.45,0.47,0.47,0.46,0.46,0.47,0.46,0.44,0.47,0.47,0.47,0.47,0.47,0.47,0.45,0.46,0.45,0.47,0.47,0.45,0.46,0.44,0.47,0.46,0.44,0.46,0.45,0.47,0.47,0.47,0.47,0.47,0.47,0.46,0.47,0.46,0.46,0.46,0.44,0.47,0.45,0.47,0.47,0.44,0.47,0.47,0.46,0.41,0.47,0.46,0.46,0.47,0.46,0.47,0.47,0.46,0.45,0.46,0.46,0.47,0.46,0.46,0.42,0.44,0.43,0.46,0.47,0.46,0.40,0.47,0.47,0.43,0.46,0.47,0.46


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.45
2005,0.46
2006,0.45
2007,0.49
2008,0.50
2009,0.45
2010,0.41
2011,0.49


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.50
3,0.51
4,0.41
5,0.38
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.37
     3          0.38
     4          0.53
     5          0.34
     6          0.30
     7          0.28
     8          0.30
     9          0.28
     10         0.42
     11         0.53
     12         0.53
2004 1          0.54
     2          0.51
     3          0.51
     4          0.40
     5          0.29
     6          0.38
     7          0.27
     8          0.41
     9          0.57
     10         0.55
     11         0.47
     12         0.48

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 99%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.99
2004,0.95
2005,1.05
2006,1.00
2007,0.99
2008,1.01
2009,1.00
2010,1.00
2011,0.99


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.95
2,1.03
3,1.05
4,1.04
5,1.00
6,1.00
7,1.01
8,1.02
9,0.97


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.86
     2                 1.12
     3                 0.96
     4                 1.02
     5                 0.92
     6                 1.01
     7                 1.06
     8                 1.04
     9                 0.93
     10                1.01
     11                1.01
     12                0.91
2004 1                 0.66
     2                 0.93
     3                 1.32
     4                 1.13
     5                 1.01
     6                 1.16
     7                 1.10
     8                 0.91
     9                 0.94
     10                1.05
     11                0.87
     12                0.61

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $425,245.23/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,538,444.84","81,340,630.21"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"9,974,248.22"
2004,"4,944,657.99"
2005,"14,866,857.44"
2006,"10,182,942.48"
2007,"7,336,998.80"
2008,"13,671,337.51"
2009,"12,785,031.25"
2010,"8,346,230.13"
2011,"7,148,491.81"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,083.33","543,083.33","543,010.42","226,624.92","7,126,218.75"
2,"495,177.08","495,177.08","495,250.00","47,500.00","5,113,845.49"
3,"542,500.00","542,864.58","542,864.58","125,372.73","5,526,562.50"
4,"525,000.00","524,635.42","524,635.42","366,598.29","3,284,387.15"
5,"542,500.00","542,500.00","542,500.00","38,000.00","9,846,418.40"
6,"525,000.00","525,000.00","525,000.00","316,245.98","4,125,845.49"
7,"542,500.00","542,500.00","542,500.00","129,862.80","11,628,454.86"
8,"542,500.00","542,500.00","542,500.00","221,740.12","12,848,468.75"
9,"525,000.00","525,000.00","525,000.00","66,500.00","4,945,312.50"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00             74,446.13       4,340,928.82  
     8                   54,250.00             14,250.00         748,437.50  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00       2,878,404.51  
2004 1                   54,250.00                  0.00       3,023,157.99  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00               0.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

<a id="utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.90,0.95


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.85,0.93
2004,1.00,1.00,1.00,0.00,0.95
2005,1.00,1.00,1.00,0.85,0.97
2006,1.00,1.00,1.00,0.85,0.96
2007,1.00,1.00,1.00,0.94,0.90
2008,1.00,1.00,1.00,0.94,0.99
2009,1.00,1.00,1.00,0.00,0.95
2010,1.00,1.00,1.00,0.94,0.97
2011,1.00,1.00,1.00,0.86,0.94


<a id="tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [24]:
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [25]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [26]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [27]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [28]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [29]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

<a id="equipment-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [30]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,363,466.15"
Repair,"20,111,549.48"
Crew Transfer,"1,160,161.47"
Site Travel,0.00
Mobilization,"7,500,000.00"
Weather Delay,"35,004,520.75"
No Requests,"14,547,289.87"
Not in Shift,"21,371,868.60"


In [31]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,363,466.15","2,363,466.15"
Repair,0.00,0.00,0.00,"20,111,549.48","20,111,549.48"
Crew Transfer,0.00,0.00,0.00,"1,160,161.47","1,160,161.47"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"7,500,000.00","7,500,000.00"
Weather Delay,0.00,0.00,0.00,"35,004,520.75","35,004,520.75"
No Requests,0.00,0.00,0.00,"14,547,289.87","14,547,289.87"
Not in Shift,0.00,0.00,0.00,"21,371,868.60","21,371,868.60"


In [32]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      60,302.08
     Repair        2,711,794.27
     Crew Transfer   119,093.75
     Site Travel           0.00
     Mobilization  1,000,000.00
     Weather Delay 2,293,403.65
     No Requests   1,412,317.71
     Not in Shift  2,377,336.76
2004 Maintenance     433,197.92
     Repair          801,517.36

In [33]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                    
1     Maintenance     206,992.19
      Repair        1,535,165.80
      Crew Transfer   106,963.54
      Site Travel           0.00
      Mobilization    500,000.00
      Weather Delay 4,245,511.20
      No Requests     593,093.75
      Not in Shift  1,794,294.27
2     Maintenance     182,929.69
      Repair        1,283,196.18

In [34]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          20,471.35
           Crew Transfer    2,861.98
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   23,351.56
           No Requests    105,546.88
           Not in Shift    12,049.48
     2     Maintenance          0.00
           Repair          28,674.48
           Crew Transfer    3,226.56
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   11,593.75
           No Requests     90,070.31
           Not in Shift    13,434.90
     3     Maintenance          0.00
           Repair          21,437.50
           Crew Transfer    2,588.54
           Site Travel          0.00

<a id="component-costs"></a>
## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [35]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"117,010,896.53"


In [36]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   56,237,719.44   
          maintenance            0.00              0.00    2,363,466.15   
          repair                 0.00              0.00   20,111,549.48   

                         total_cost  
component action                     
turbine   delay       56,237,719.44  
          maintenance  2,363,466.15  
          repair      20,111,549.48

In [37]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"3,392,000.00",0.00,"7,543,703.13","10,935,703.13"
2004,turbine,"3,113,000.00",0.00,"4,803,983.51","7,916,983.51"
2005,turbine,"5,124,000.00",0.00,"12,923,499.37","18,047,499.37"
2006,turbine,"4,611,500.00",0.00,"8,127,294.27","12,738,794.27"
2007,turbine,"2,949,000.00",0.00,"4,098,582.13","7,047,582.13"
2008,turbine,"4,343,000.00",0.00,"11,731,519.10","16,074,519.10"
2009,turbine,"3,560,500.00",0.00,"8,787,591.15","12,348,091.15"
2010,turbine,"3,170,500.00",0.00,"6,857,330.73","10,027,830.73"
2011,turbine,"2,823,500.00",0.00,"4,227,046.01","7,050,546.01"


In [38]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"3,238,000.00",0.00,"7,880,395.75","11,118,395.75"
2,turbine,"2,111,000.00",0.00,"5,647,730.90","7,758,730.90"
3,turbine,"2,595,000.00",0.00,"6,648,359.71","9,243,359.71"
4,turbine,"2,648,000.00",0.00,"3,291,394.00","5,939,394.00"
5,turbine,"4,004,500.00",0.00,"8,277,418.40","12,281,918.40"
6,turbine,"2,892,000.00",0.00,"3,037,262.15","5,929,262.15"
7,turbine,"5,442,500.00",0.00,"10,470,315.10","15,912,815.10"
8,turbine,"4,727,500.00",0.00,"10,620,206.26","15,347,706.26"
9,turbine,"2,625,500.00",0.00,"5,829,263.02","8,454,763.02"


In [39]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         14,000.00              0.00       57,203.13   
     2     turbine         67,000.00              0.00       56,929.69   
     3     turbine         20,000.00              0.00       28,656.25   
     4     turbine        114,500.00              0.00       66,828.13   
     5     turbine         20,000.00              0.00       30,296.88   
     6     turbine         40,500.00              0.00       33,869.79   
     7     turbine      1,929,500.00              0.00    3,939,996.53   
     8     turbine        408,000.00              0.00      585,294.27   
     9     turbine         57,000.00              0.00       53,958.33   
     10    turbine         68,500.00              0.00       61,505.21   
     11    turbine        186,500.00              0.00      111,963.54   
     12    turbine        466,500.00              0.00    2,517,201.39   
2004 1     turbine        868,000.00              0.00    3,185,032.99   
     2     turbine        200,000.00              0.00      151,375.00   
     3     turbine        226,500.00              0.00      161,875.00   
     4     turbine        266,500.00              0.00      154,000.00   
     5     turbine        241,000.00              0.00      161,875.00   
     6     turbine        234,000.00              0.00      155,750.00   
     7     turbine        169,000.00              0.00      127,276.04   
     8     turbine        150,500.00              0.00      101,244.79   
     9     turbine        155,500.00              0.00      127,804.69   
     10    turbine        256,500.00              0.00      161,875.00   
     11    turbine        145,500.00              0.00      154,000.00   
     12    turbine        200,000.00              0.00      161,875.00   
2005 1     turbine        182,500.00              0.00      161,875.00   
     2     turbine        188,500.00              0.00      408,220.49   
     3     turbine        966,000.00              0.00    4,832,455.10   
     4     turbine        706,500.00              0.00      376,041.67   

                       total_cost  
year month component               
2003 1     turbine      71,203.13  
     2     turbine     123,929.69  
     3     turbine      48,656.25  
     4     turbine     181,328.13  
     5     turbine      50,296.88  
     6     turbine      74,369.79  
     7     turbine   5,869,496.53  
     8     turbine     993,294.27  
     9     turbine     110,958.33  
     10    turbine     130,005.21  
     11    turbine     298,463.54  
     12    turbine   2,983,701.39  
2004 1     turbine   4,053,032.99  
     2     turbine     351,375.00  
     3     turbine     388,375.00  
     4     turbine     420,500.00  
     5     turbine     402,875.00  
     6     turbine     389,750.00  
     7     turbine     296,276.04  
     8     turbine     251,744.79  
     9     turbine     283,304.69  
     10    turbine     418,375.00  
     11    turbine     299,500.00  
     12    turbine     361,875.00  
2005 1     turbine     344,375.00  
     2     turbine     596,720.49  
     3     turbine   5,798,455.10  
     4     turbine   1,082,541.67

<a id="fixed-costs"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [40]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [41]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [42]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [44]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [45]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"252,521.73","79,724.75","72,178.25",544.00
major repair,"71,766.26","1,961.36","1,469.71",29.00
major replacement,"165,255.37","10,782.12","165,255.37",59.00
manual reset,"245,528.68","37,492.25","12,989.75","5,810.00"
medium repair,"16,226.67","11,819.75","9,093.75",238.00
minor repair,"78,392.55","31,534.25","14,381.25","2,217.00"


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [46]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,634,508,357.50"


In [47]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,634,508,357.50","119,025,113.00","123,632,561.00","119,815,115.50","117,094,647.50","119,086,063.00","117,971,302.50","123,118,528.00","122,463,158.50","121,357,009.00","120,277,698.00","123,288,501.00","119,835,928.00","116,004,839.00","122,994,674.50","122,485,093.50","122,418,004.50","123,161,932.50","122,649,536.50","122,466,992.00","118,605,634.50","119,999,274.00","118,846,530.00","122,857,475.50","123,216,768.00","117,484,144.00","121,767,013.50","116,190,364.50","122,585,245.00","121,834,789.50","116,092,098.50","120,525,303.00","118,135,096.00","123,209,029.50","122,782,782.50","122,737,522.00","123,059,024.00","123,253,630.50","123,061,651.00","121,681,092.50","123,905,950.50","120,061,232.50","121,884,528.50","121,283,255.00","115,401,366.50","122,891,059.00","117,995,285.50","122,487,360.00","123,514,415.00","115,848,577.00","122,591,890.00","122,600,044.00","120,656,658.00","107,693,210.50","123,520,031.50","122,227,366.50","121,195,391.00","123,714,621.00","121,751,420.00","122,536,919.50","122,902,989.50","121,813,961.50","118,656,997.50","119,793,883.00","120,647,279.00","122,938,012.00","120,990,042.00","120,122,173.50","110,874,816.50","116,661,199.00","112,433,901.00","121,941,516.00","122,700,028.00","122,219,045.00","104,426,617.50","122,731,166.00","122,949,611.50","114,137,229.50","120,040,526.00","122,819,404.00","121,876,212.50"


In [48]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"854,704,187.50"
2004,"942,757,714.50"
2005,"964,042,664.00"
2006,"954,600,985.50"
2007,"1,032,190,210.50"
2008,"1,056,917,932.50"
2009,"955,814,829.00"
2010,"863,354,699.50"
2011,"1,021,670,401.50"


In [49]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,073,686,555.00"
2,"819,610,161.00"
3,"902,155,773.00"
4,"709,477,316.50"
5,"684,301,278.50"
6,"555,092,755.00"
7,"581,989,396.00"
8,"644,698,767.50"
9,"798,580,384.50"


In [50]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     111,942,136.50
     2      59,121,715.50
     3      66,995,691.50
     4      91,053,160.50
     5      60,264,211.50
     6      51,123,378.00
     7      50,860,030.00
     8      53,739,829.00
     9      47,695,258.50
     10     75,430,096.00
     11     91,324,555.50
     12     95,154,125.00
2004 1      96,844,632.50
     2      85,772,125.50
     3      91,614,487.50
     4      68,588,723.50
     5      52,325,907.00
     6      65,264,095.00
     7      48,138,037.00
     8      72,697,778.00
     9      97,959,322.00
     10     97,323,398.50
     11     81,198,966.00
     12     85,030,242.00

<a id="pysam"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [51]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [53]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [54]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [55]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
